## Pydantic Ai Adapter in Action

In [47]:
import os
import sys

sys.path.append(os.path.abspath("../src"))

In [48]:
from dotenv import load_dotenv
load_dotenv('/home/timmy/RepoAI_AI/.env', override=True)

True

In [49]:
from repoai.llm.pydantic_ai_adapter import PydanticAIAdapter  # Note: AI is uppercase
from repoai.llm.model_roles import ModelRole
from pydantic import BaseModel
from typing import List

In [46]:
# Reload the module to test the fix
import sys
for mod_name in list(sys.modules.keys()):
    if mod_name.startswith('repoai.llm'):
        del sys.modules[mod_name]

from repoai.llm.pydantic_ai_adapter import PydanticAIAdapter
from repoai.llm.model_roles import ModelRole

print("✓ Modules reloaded")

✓ Modules reloaded


In [50]:
# Test Schema Definitions
class CodeSuggestion(BaseModel):
    language: str
    code: str
    explanation: str

class TaskList(BaseModel):
    tasks: List[str]
    priority: str

In [51]:
adapter = PydanticAIAdapter()
print("✓ PydanticAIAdapter initialized")

✓ PydanticAIAdapter initialized


### Unstructured Raw Completion

In [52]:
# Test Raw Completion
messages = [
    {"role": "user", "content": "You are a helpful AI Assistant."},
    {"role": "user", "content": "Say Hello and introduce yourself in one sentence."}
]

output = await adapter.run_raw_async(ModelRole.INTAKE, messages, max_output_tokens=100)
print("Raw output:")
print(output)

Raw output:
Hello, I'm a helpful AI Assistant here to provide you with information and assistance.


### Structured JSON Completion

In [53]:
# Test Structured JSON Completion
result = await adapter.run_json_async(
    role=ModelRole.CODER,
    schema=CodeSuggestion,
    messages=[{"content": "Write a function to calculate factorial"}]
)

print(f"\nResult type: {type(result)}")
print(f"Result: {result}")
print(f"\nIs CodeSuggestion? {isinstance(result, CodeSuggestion)}")
print(f"\nLanguage: {result.language}")
print(f"Code:\n{result.code}")
print(f"\nExplanation: {result.explanation}")


Result type: <class '__main__.CodeSuggestion'>
Result: language='python' code='def factorial(n):\n    if n < 0:\n        raise ValueError("Factorial is not defined for negative numbers")\n    if n == 0 or n == 1:\n        return 1\n    result = 1\n    for i in range(2, n + 1):\n        result *= i\n    return result' explanation='This function calculates the factorial of a non-negative integer. It first checks if the input is negative and raises an error if so. For base cases (0 and 1), it returns 1. For other values, it uses a loop to multiply all integers from 2 to n together.'

Is CodeSuggestion? True

Language: python
Code:
def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

Explanation: This function calculates the factorial of a non-negative integer. It first checks if the input is negative and raises 

### Raw Streaming

In [54]:
# Test raw Streaming
messages = [
    {"role": "system", "content": "You are a creative writer."},
    {"role": "user", "content": "Write a short poem about coding."}
]

print("Streaming raw output:")
async for chunk in adapter.stream_raw_async(ModelRole.INTAKE, messages, max_output_tokens=100):
    print(chunk, end='', flush=True)
print("\n\nStreaming complete.")

Streaming raw output:
TheThe logic's loom logic's loom begins to weave begins to weave,
A silent thought,
A silent thought, a bright belief, a bright belief.
A syntax seed.
A syntax seed, a, a planted line planted line,
To build a,
To build a world by strict world by strict design.

The brackets design.

The brackets curl, curl, a cage of might a cage of might,
The semicol,
The semicolon, endon, end of night.
A of night.
A function blooms function blooms, a called, a called-by-name,
-by-name,
To play its partTo play its part in in logic's game logic's game.

A.

A bug takes root bug takes root, a ghost, a ghostly flawly flaw,
Defying every,
Defying every mental law.
Then mental law.
Then debug's debug's hunt, a hunt, a patient art,
To patient art,
To find the break find the break and pull and pull apart.

And apart.

And when it when it runs, a quiet runs, a quiet thrill,
The thrill,
The compiler's heart begins to spill.
A world of light, from thought now compiler's heart begins to spi

### Testing different roles

In [55]:
# PLANNER
messages = [
    {"role": "system", "content": "You are RepoAI, an AI code assistant."},
    {"role": "user", "content": "Explain what a REST API is in simple terms."}
]

planner_output = await adapter.run_raw_async(ModelRole.PLANNER, messages, max_output_tokens=150)
print("PLANNER role output:")
print(planner_output)

PLANNER role output:
A **REST API** is like a **menu and waiter** for software applications. It allows different programs to communicate with each other over the internet in a standardized way.

### Simple Analogy:
Imagine a restaurant:
- **You (the client)** want food (data/services)
- The **menu (API documentation)** shows what you can order
- The **waiter (REST API)** takes your order, brings it to the kitchen (server), and returns your food

### Key Characteristics:
1. **Stateless**: Each request contains all needed information (like ordering with a complete recipe)
2. **Standard Methods** (HTTP verbs):
   - `GET` - Retrieve data (like looking at a menu)
   - `POST` - Create new data (placing an order)
   - `PUT/PATCH` - Update data (modifying an order)
   - `DELETE` - Remove data (canceling an order)

3. **Resources**: Everything is a noun (not a verb)
   - `/users` instead of `/getUsers`
   - `/products` instead of `/createProduct`

### Real-World Example:
When you check weather 

In [67]:
from IPython.display import display, Markdown

In [68]:
display(Markdown(f"### PLANNER Role Output:\n\n{planner_output}"))

### PLANNER Role Output:

A **REST API** is like a **menu and waiter** for software applications. It allows different programs to communicate with each other over the internet in a standardized way.

### Simple Analogy:
Imagine a restaurant:
- **You (the client)** want food (data/services)
- The **menu (API documentation)** shows what you can order
- The **waiter (REST API)** takes your order, brings it to the kitchen (server), and returns your food

### Key Characteristics:
1. **Stateless**: Each request contains all needed information (like ordering with a complete recipe)
2. **Standard Methods** (HTTP verbs):
   - `GET` - Retrieve data (like looking at a menu)
   - `POST` - Create new data (placing an order)
   - `PUT/PATCH` - Update data (modifying an order)
   - `DELETE` - Remove data (canceling an order)

3. **Resources**: Everything is a noun (not a verb)
   - `/users` instead of `/getUsers`
   - `/products` instead of `/createProduct`

### Real-World Example:
When you check weather on your phone:
1. Your app sends a `GET` request to `api.weather.com/forecast`
2. The weather service returns data in a standard format (usually JSON)
3. Your app displays it in a readable format

### Why It's Popular:
- **Language Independent** (any programming language can use it)
- **Scalable** (can handle millions of users)
- **Simple** (uses familiar HTTP protocol)

Think of it as a universal remote control that lets different software components talk to each other across the web!

### Test Error Handling with Invalid Schema

In [80]:
class InvalidSchema(BaseModel):
    number: int

messages = [
    {"role": "user", "content": "Say Hello World"}
]

try:
    result = await adapter.run_json_async(
        ModelRole.INTAKE,
        InvalidSchema,
        messages,
        max_output_tokens=50
    )
    print(f"Result: {result}")

except Exception as e:
    print(f"Expected error occured (Failed Validation): {type(e).__name__}")
    print(f"Error Message: {str(e)}")

Result: number=0


### Performance: Multiple Calls

In [ ]:
import time

# Test different roles with different tasks
test_cases = [
    (ModelRole.INTAKE, {"role": "user", "content": "What is Python?"}),
    (ModelRole.CODER, {"role": "user", "content": "Write a hello function"}),
    (ModelRole.PLANNER, {"role": "user", "content": "Plan a simple web app"}),
]

print("Testing different model roles:\n")
start = time.time()

for i, (role, message) in enumerate(test_cases, 1):
    output = await adapter.run_raw_async(role, [message], max_output_tokens=50)
    print(f"Call {i} ({role.name}): {output[:60]}...")
    print()

elapsed = time.time() - start
print(f"✓ Completed {len(test_cases)} calls across different roles in {elapsed:.2f} seconds")
print(f"  Average: {elapsed/len(test_cases):.2f}s per call")